# Исследование надёжности заёмщиков

<a id="head"></a>

**Учебная работа. Самостоятельный исследовательский проект в рамках курса "Аналитик данных" Яндекс.Практикума.  
Спринт 1**

**Описание задания:**  
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель:** провести анализ данных о платёжеспособности клиентов и выделить категории клиентов, для которых вероятность возвращения кредита вовремя наиболее вероятна и наименее вероятна.

**Задачи:**  
1. Подготовить данные: найти и удалить дублирующиеся данные, найти и обработать пропущенные значения, найти и исправить ошибки в данных.
2. Выделить категории клиентов (заёмщиков), по которым будет проводиться анализ.
3. Проанализировать данные по выбранным категориям клиентов.
4. Описать и оформить результаты.

**Данные:**  
Данные о платёжеспособности клиентов получены в виде файла data.csv.

**Описание данных:**  
`children` — количество детей в семье  
`days_employed` — общий трудовой стаж в днях  
`dob_years` — возраст клиента в годах  
`education` — уровень образования клиента  
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose` — цель получения кредита

**Содержание:**

1. [Гипотезы](#hypothesis)
2. [Загрузка и обзор данных](#open)
3. [Предобработка данных](#preprocessing)
    - [Поиск и обрабока пропущенных значений](#isna)
    - [Замена типа данных](#type)
    - [Обработка дубликатов](#duplicates)
    - [Лемматизация](#lemmas)
    - [Категоризация данных](#categories)
        * [Категоризация целей](#purpose_cat")
        * [Категоризация доходов заёмщиков](#income_cat)
        * [Категоризация по наличию детей](#children_cat)
        * [Категоризация по семейному статусу](#family_cat)
    - [Подготовка финальной таблицы для анализа](#final_data)
4. [Анализ данных о надёжности заёмщика](#analise) 
    - [Наличие и количество детей](#children_factor)
    - [Семейное положением](#family_factor)
    - [Уровень дохода](#income_factor)
    - [Цель кредита](#purpose_factor)
5. [Вывод](#Conclusion)



## Гипотезы:
<a id="hypothesis"></a>

**Количество детей**

*Гипотеза 1:* Вовремя чаще возвращают кредиты заёмщики, у которых нет детей совсем, так как они не обременены лишними расходами.  
*Гипотеза 2:* Чем больше у заёмщика детей, тем больше вероятность, что он не вернёт кредит вовремя.  
*Гипотеза 3:* Наличие детей по разному влияет на вероятность вернуть кредит вовремя у заёмщиков в браке и не в в браке.

**Семейное положение**

*Гипотеза 1:* Вовремя чаще возвращают кредиты заёмщики, у которых есть семья (партнёр), так как им есть у кого попросить помощи, чесли что-то пойдёт не по плану.  
*Гипотеза 2:* Вовремя чаще возвращают кредиты заёмщики в браке.  
*Гипотеза 3:* Семейный статус по разному влияет на вероятность вернуть кредит вовремя у мужчин и у женщин.

**Доход**

*Гипотеза 1:* Чем выше доход заёмщика, тем больше вероятность, что он вернёт доход вовремя.  
*Гипотеза 2:* Вовремя чаще возвращают кредиты заёмщики со средним доходом.

**Цели кредита**

*Гипотеза 1:* Вовремя чаще возвращают небольшие и краткосрочные кредиты (например, на ремонт, на свадьбу или на автомобиль)

## Загрузка и обзор данных
<a id="open"></a>

**Импортируем библиотеку Pandas и открываем файл с данными:**

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

**Просматриваем общую информацию по колонкам:**

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В нужных для ответа на поставленный банком вопрос столбцах (дети, семейное положение и наличие долга) пропусков нет.
Но есть одинаковое количество пропусков в столбцах `days_employed` и `total_income`, это наводит на мысль о том, что пропуски возникли не случайно, а как-то взаимосвязаны.

Странности, касающиеся типа данных в столбцах: 
 - Стаж `days_employed` почему-то имеет тип float, хотя единица измерения - дни, это странно. 


<div class="alert alert-success" style="border-radius: 15px">
<h2> Комментарий ревьюера</h2>

Верно подмечено.
</div>

**Просматриваем первые строки таблицы:**

In [3]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


1. Аномалий в данных, которые интересуют нас больше всего (наличие долга, количество детей и семейное положение), на первый взгляд не видно.

Что касается остальных данных, то они есть:

2. Что-то не то с данными о стаже (столбец `days_employed`). Большая часть значений в этом столбце отрицательные (стаж не может быть отрицательным), а часть - неправдоподобно большая (338551 дней или 400281 дней - в переводе в годы это больше ста лет). Значения в столбце имеют по 6 знаков после запятой. Скорее всего, данные во всём столбце были обработаны или присоединены к таблице с ошибкой. Если вдруг они понадобятся для анализы, то нужно будет обращаться к источнику данных, потому что пока нет предположений, какая именно операция привела их такому виду.


3. В столбце `education` встречаются одни и те же категории, записанные в разном регистре. 

**Вывод**

В данных есть аномалии. 
1. Столбец `days_employed` содержит странные данные. В текущей работе он не будет нужен, но в дальнейшем нужно будет разобраться, почему данные пришли в таком формате. 
2. Нужно поменять тип данных столбца `days_employed` на целочисленный.
3. Стоит обратить внимание на одинаковое количество пропусков в столбцах `days_employed` и `total_income` и проверить, взаимосвязаны ли они. Если удастся, выяснить, с чем связано возникновение пропусков.
4. Нужно обратить внимание на написание категорий образования (столбец `education`)при удалении дублирующихся значений.


## Предобработка данных
<a id="preprocessing"></a>

### Поиск и обработка пропущенных значений
<a id="isna"></a>

Количество пропусков однаковое для столбцов со стажем и с доходом клиентов, что наводит на мысль, что пропуски в одних и тех же строках. Чтобы это проверить, отбираем и выводим на экран только строки с пропусками в столбце доход:

In [4]:
data_na = data[data['total_income'].isna()]
display(data_na.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


В первых десяти строках пропуски действительно совпадают. Чтобы проверить, так ли это во всей остальной таблице, сравним количество пропусков в таблице с пропусками по столбцу доход и количество пропусков в исходной таблице:

In [5]:
print('Количество пропусков в столбце доход в исходной таблице:', data['total_income'].isna().sum())
print('Количество пропусков в столбце стаж в исходной таблице:', data['days_employed'].isna().sum())
print('Количество пропусков в столбце доход в таблице с пропусками:', data_na['total_income'].isna().sum())
print('Количество пропусков в столбце стаж в таблице с пропусками:', data_na['days_employed'].isna().sum())
print('Количество строк в таблице с пропусками:', len(data_na))

Количество пропусков в столбце доход в исходной таблице: 2174
Количество пропусков в столбце стаж в исходной таблице: 2174
Количество пропусков в столбце доход в таблице с пропусками: 2174
Количество пропусков в столбце стаж в таблице с пропусками: 2174
Количество строк в таблице с пропусками: 2174


Количество пропусков в исходной таблице совпадает с количеством пропусков и с количеством строк в таблице с пропусками. Все пропуски точно возникают в одних и тех же строках. Чтобы понять, как лучше заполнить пропуски, попробуем найти закономерность, по которой они возникают. Для этого сравним частоту встречаемости разных категорий клиентов в группе с пропусками и во всём массиве данных.

Функция для поиска закономерностей возникновения пропусков:

In [6]:
def compare_data(data1, data2, column):
    # функция для сравнения частоты встречаемости значений в одинаковых столбцах двух сходных по структуре таблиц.
    # В качестве аргументов функция принимает имена двух таблиц и название сравниваемого столбца.
    # Возвращает частоту встречаемости значений в столбце первой и второй таблицы
    
    value_counts1 = data1.groupby(column)[column].count()
    value_counts2 = data2.groupby(column)[column].count()
    comparison = [value_counts1, value_counts2]
    return comparison

Применим функцию compare_data к столбцу `children`:

In [7]:
display(compare_data(data, data_na, 'children'))


[children
 -1        47
  0     14149
  1      4818
  2      2055
  3       330
  4        41
  5         9
  20       76
 Name: children, dtype: int64,
 children
 -1        3
  0     1439
  1      475
  2      204
  3       36
  4        7
  5        1
  20       9
 Name: children, dtype: int64]

Порядок значений отличается, но пропорции количества детей в данных примерно те же, что и в полных данных. Зато выявилась аномалия: у части клиентов записан '-1' ребёнок, а у 76 человек - '20'. Скорее всего, что '-1' возникло на месте '1', а '20' - на месте '2'. 

Есть ли различия в образовании:

In [8]:
print(compare_data(data, data_na, 'education_id'))

[education_id
0     5260
1    15233
2      744
3      282
4        6
Name: education_id, dtype: int64, education_id
0     544
1    1540
2      69
3      21
Name: education_id, dtype: int64]


Значения тоже отличаются на порядок, но разницы в пропорциях нет. По семейному статусу:

In [9]:
display(compare_data(data, data_na, 'family_status'))

[family_status
 Не женат / не замужем     2813
 в разводе                 1195
 вдовец / вдова             960
 гражданский брак          4177
 женат / замужем          12380
 Name: family_status, dtype: int64,
 family_status
 Не женат / не замужем     288
 в разводе                 112
 вдовец / вдова             95
 гражданский брак          442
 женат / замужем          1237
 Name: family_status, dtype: int64]

В семейном статусе различий нет. По полу:

In [10]:
display(compare_data(data, data_na, 'gender'))

[gender
 F      14236
 M       7288
 XNA        1
 Name: gender, dtype: int64,
 gender
 F    1484
 M     690
 Name: gender, dtype: int64]

По полу нет различий. По должности:

In [11]:
display(compare_data(data, data_na, 'income_type'))

[income_type
 безработный            2
 в декрете              1
 госслужащий         1459
 компаньон           5085
 пенсионер           3856
 предприниматель        2
 сотрудник          11119
 студент                1
 Name: income_type, dtype: int64,
 income_type
 госслужащий         147
 компаньон           508
 пенсионер           413
 предприниматель       1
 сотрудник          1105
 Name: income_type, dtype: int64]

По должности тоже не видно явных различий. Мы визуально оценили зависимость возникновения пропусков от количества детей, семейного статуса, пола и должности, и не нашли никакой связи, поэтому будем считать возникновение пропусков случайным. Оценим количество пропусков, чтобы решить, можно ли избавиться от них исключением строк:

In [12]:
na_prop = len(data_na)/len(data)
print('Доля пропусков: {:.0%}'.format(na_prop))

Доля пропусков: 10%


10% пропусков - это слишком много для удаления строк, поэтому придётся заменять значения, несмотря на то, что мы не знаем, в связи с чем пропуски возникли. Для стажа и для дохода наиболее адекватной будет замена пропусков медианой (0 и среднее может сдвинуть распределение в одну из сторон) по категорям клиентов.

Так как трудовой стаж может влиять на доход, сначала исследуем данные о стаже.
В столбце `days_employed` данные выглядят неправдоподобными: часть данных имеет отрицательное значение, а часть - слишком большое значение. Посмотрим на эти две части данных по отдельности.  
Сначала изучим отрицательные значения стажа.  
Для начала попробуем домножить их на -1 и разделить на 365, чтобы получить положительные значения в годах. Сохраним их в столбец `years_employed`.
Выведем данные о возрасте и стаже в виде сводной таблицы: сравним минимальные, медианные и максимальные значения стажа с возрастом, чтобы оценить правдоподобность значений стажа.   

In [13]:
data_minus = data.loc[data['days_employed'] <= 0].copy()

data_minus['years_employed'] = data_minus['days_employed'] / -365

data_pivot_minus = data_minus.pivot_table(
    index=['dob_years'],  
    values='years_employed', 
    aggfunc=['min', 'median', 'max', 'count'])


display(data_pivot_minus)


,min,median,max,count
,years_employed,years_employed,years_employed,years_employed
dob_years,,,,
0,0.298540,4.276440,29.285618,74
19,0.306199,1.984911,2.795022,13
20,0.305760,1.848874,4.240559,46
21,0.141087,1.695161,5.339534,93
22,0.249748,1.915236,5.771288,165
23,0.192032,1.890970,7.045418,218
24,0.269348,2.596523,9.274298,243
25,0.221324,2.518354,10.675904,334


Видно ещё один вид пропусков: В столбце `dob_years` у 74 клиентов стоит 0. Если бы значения стажа были реалистичными, то можно было бы восстановить возраст, прибавив к стажу 18 (или медиану разницы возраста и стажа остальных клиентов). В нашем случае лучше заменить пропущенные значения возраста на медиану по возрасту, чтобы избежать возникновения других странных значений.

Посмотрим на распределение величины стажа в зависимости от возраста.

- Максимальные значения стажа выглядят довольно правдоподобно. Они равномерно растут вместе с возрастом. Правда, получается, что многие клиенты начали работать в 15-16 лет, но это вполне возможно, если выйти на работу после 9 класса, на то он и максимальный стаж.  
- Минимальные значения стажа возможны, но маловероятны. Довольно трудно представить себе, что так много людей, берущих кредиты, имеет трудовой стаж меньше года.  
- Медианные значения стажа совсем неправдоподобны. Получается, что половина клиентов вплоть до пенсионного возраста имеет стаж меньше 6 лет.
 


Так как нам нужно заполнить пропуски, а сами значения стажа для анализа не потребуется, в данном случае можно счесть их приемлемыми.


Посмотрим на вторую группу странных значений (слишком большой стаж):

In [14]:
data_huge = data.loc[data['days_employed'] > 0].copy()
data_huge['years_employed'] = data_huge['days_employed'] / 365

In [15]:
data_pivot_huge = data_huge.pivot_table(
    index=['dob_years'],  
    values='years_employed', 
    aggfunc=['min', 'median', 'max', 'count'])

display(data_pivot_huge)

,min,median,max,count
,years_employed,years_employed,years_employed,years_employed
dob_years,,,,
0,908.379592,1002.925427,1098.609249,17
22,917.162356,917.162356,917.162356,1
26,998.214239,1032.527897,1066.841555,2
27,903.511521,1032.396126,1039.704390,3
28,959.837699,959.837699,959.837699,1
31,924.724567,924.724567,924.724567,1
32,900.896837,921.132558,967.276850,3
33,913.408222,1001.779458,1090.150694,2


В этой группе значений не видно никакой логики. Значения мало меняются в зависимости от возраста (минимальные значения для всех возрастов примерно около 900, а максимальные - около 1100). Значений для многих возрастов нет, основная масса клиентов из этой группы старше 40-50 лет.
Для этой группы клиентов сложно восстановить данные, поэтому примем для них значение стажа на 18 лет меньше возраста.

***Возраст***  
Для этого сначала заполним пропущенные значения в столбце возраст. Так как достоверно восстановить данные о стаже не удалось, используем медиану по выборке.

In [16]:
dob_years_median = data['dob_years'].median()
print('dob_years_median =', dob_years_median)

dob_years_median = 42.0


Заменяем нулевые значения на значение медианы и распечатываем минимальный возраст для проверки:

In [17]:
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_median
print('dob_years_min =', data['dob_years'].min())

dob_years_min = 19.0


***Стаж***  
Теперь исправим значения стажа. Напишем функцию days_to_years. Она будет принимать в качестве аргумента строку таблицы со стажем в днях и возврастом и возвращать стаж в годах. 
 - если значение `days_employed` отсутствует (NaN), функция будет возвращать значение (возраст -18),
 - если значение `days_employed` больше нуля, так же (возраст -18),
 - если значение `days_employed` меньше нуля, то значение (`days_employed` * (-1) / 365)

In [18]:
def days_to_years(row):
    """Функция переводит дни в годы для отрицательных значений длительности стажа или 
    присваивает значение на 18 лет меньше для остальных случаев (NaN или слишком больших)"""
    
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    if pd.isna(days_employed):
        years = dob_years - 18
    elif days_employed > 0:
        years = dob_years - 18
    else:
        years = days_employed / -365
    return years

Применяем функцию и проверяем, остались ли пропуски в столбце `years_employed`:

In [19]:
data['years_employed'] = data.apply(days_to_years, axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
years_employed      21525 non-null float64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


Проверяем, каковы минимальное, медианное и максимальное значение стажа в новом столбце `years_employed`:

In [20]:
print('years_employed_min = {:.2f}'.format(data['years_employed'].min()))
print('years_employed_median = {:.2f}'.format(data['years_employed'].median()))
print('years_employed_max = {:.2f}'.format(data['years_employed'].max()))

years_employed_min = 0.07
years_employed_median = 7.01
years_employed_max = 56.00


Как и ожидалось, медианное значение вызывает сомнение, а минимальное и максимальное вполне правдоподобны. Это уже лучше, чем отрицательные значения, и пропуски заполнены. Удаляем столбец `days_employed` и делаем проверку:

In [21]:
data = data.drop('days_employed', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
years_employed      21525 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Столбец `years_employed` есть, столбца `days_employed` нет. Пропусков d lfyysв данных о стаже нет.

***Доход***  
Пропущенные значения стажа восстановим, приписав им значение медианного дохода сходных по характеристикам групп клиентов. Скорее всего, на доход должны сильно влиять тип занятости и пол. Посмотрим, насколько большой разброс значений дохода для разных клиентов по этим категорям.

*Тип занятости*  
Создаём сводную таблицу, в которой будет отображён медианый доход для разных типов занятости.

In [22]:
data_pivot_income_type = data.pivot_table(
    index='income_type', values='total_income', aggfunc=['median', 'count']
)
display(data_pivot_income_type.sort_values(by=('median', 'total_income')))

,median,count
,total_income,total_income
income_type,,
в декрете,53829.130729,1
студент,98201.625314,1
пенсионер,118514.486412,3443
безработный,131339.751676,2
сотрудник,142594.396847,10014
госслужащий,150447.935283,1312
компаньон,172357.950966,4577
предприниматель,499163.144947,1


У основной массы разброс доходов примерно раза в полтора. Тип занятости определённо стоит учитывать при восстановлении пропущенных значения дохода.  

*Гендер*  
Проверим, есть ли различия в доходе в зависимости от гендера:

In [23]:
data_pivot_income_gender = data.pivot_table(
    index='gender', values='total_income', aggfunc=['median', 'count']
)
display(data_pivot_income_gender.sort_values(by=('median', 'total_income')))

,median,count
,total_income,total_income
gender,,
F,134155.283479,12752
M,167714.343716,6598
XNA,203905.157261,1


Похоже, мужчины в среднем зарабатывают больше.  
Кроме этого есть странная запись о гендере "XNA". Вряд ли можно по остальным данным понять, о ком идёт речь. Такая строка всего одна, поэтому можно её удалить без ущерба для точности анализа. Удаляем строку и делаем проверку. 

In [24]:
data = data.drop(data[data['gender'] == 'XNA'].index)
print(data['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


Гендер тоже стоит учитывать при восстановлении пропущенных значений дохода. Посмотрим, что получится, если разбить клиентов на группы по типу занятости и гендеру:

In [25]:
data_pivot_income_gender_type = data.pivot_table(
    index='income_type', columns='gender', values='total_income', aggfunc=['median', 'count']
)

display(data_pivot_income_gender_type)

median                  count        
gender                       F              M       F       M
income_type                                                  
безработный      202722.511368   59956.991984     1.0     1.0
в декрете         53829.130729            NaN     1.0     NaN
госслужащий      136982.489425  185964.946748   962.0   350.0
компаньон        160820.776207  196818.800823  2870.0  1706.0
пенсионер        115807.789733  130739.759955  2806.0   637.0
предприниматель  499163.144947            NaN     1.0     0.0
сотрудник        130615.610597  162161.177474  6111.0  3903.0
студент                    NaN   98201.625314     NaN     1.0

Группы получились неравномерные. В каких-то группах оказалось по одному человеку, а в каких-то - по некольку тысяч. Однако больших группах получился хороший разброс по доходу: от 130 тысяч до 196.   
Можно было бы добавить возраст, но тогда группы станут слишком дробными, так что оставим так. Заменим в каждой из групп пропущенные значения на медиану по группе. Для этого пишем цикл для группированных данных:

In [26]:
for name, data_grouped in data.groupby(['income_type', 'gender']):
    """Считаем медиану для группы"""
    mediana = data_grouped['total_income'].median()
    """Находим в исходной таблице клиентов из группы и заменяем пропущенные значения на медиану"""
    data.loc[
        ((data['income_type'] == name[0]) & 
        (data['gender'] == name[1]))
        , 'total_income'] = data['total_income'].fillna(mediana)

/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


При работе этого блока возникает предупреждение о возможной ошибке. Проверим, остались ли пропуски:

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21523 non-null float64
purpose             21524 non-null object
years_employed      21524 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 2.8+ MB


Остался всего один пропуск. Наверное, это какой-нибудь предприниматель или студент, которого не с кем было сравнить.  
Проверим, правильно ли прошла замена. Должно получиться несколько групп с одинаковыми значениями дохода:

In [28]:
print(data['total_income'].value_counts().sort_values().tail(10))

401469.373495      1
303202.200463      1
185964.946748     35
130739.759955     62
136982.489425    112
196818.800823    181
160820.776207    327
115807.789733    352
162161.177474    413
130615.610597    694
Name: total_income, dtype: int64


Да, похоже на правду. Посмотрим, какой пропуск не удалось заполнить.

In [29]:
display(data.loc[data['total_income'].isna()])

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
5936,0,58.0,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,40.0


Женатый предприниматель 58 лет без долгов. У нас во всей выборке два или три предпринимателя, и один из них - без данных о доходе. Делать обобщения по такому количесву данных слишком рискованно. Бизнес может быть доходным, а может быть убыточным, дохд может быть большим, а может и не быть.
С другой стороны, тип занятости мы не планируем анализировать как фактор, влияющий на вероятность вернуть кредит вовремя. Чтобы не удалять строку, заменим пропуск на медианный доход для мужчин.

In [30]:
for name, data_grouped in data.groupby(['gender']):
    """Считаем медиану для группы"""
    mediana = data_grouped['total_income'].median()
    """Находим в исходной таблице клиентов из группы и заменяем пропущенные значения на медиану"""
    data.loc[
        (data['gender'] == name)
        , 'total_income'] = data['total_income'].fillna(mediana)

Проверка:

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
years_employed      21524 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 2.8+ MB


Пропусков в данных нет.

**Вывод**

Пропуски в столбцах  `days_employed` и `total_income` действительно оказались взаимосвязаны, но с чем связано их возникновение, выяснить не удалось. Кроме этого обнаружились нулевые значения в столбце `dob_years`.
- Нулевые значения в столбце `dob_years` заменены на медианный возраст.
- Отрицательные значения в столбце `days_employed` переведены в положительные и в годы, создан столбец `years_employed`
- Слишком большие значения и пропуски в столбце `days_employed` заменены на значение, на 18 лет меньше возраста, и сохранены в столбец `years_employed`
- Пропущенные значения дохода (столбец `total_income`) заменены на медианные значения по группам клиентов.


### Замена типа данных
<a id="type"></a>

В столбце `debt` (данные о неневозращении кредита в срок) хранятся данные в формате 1/0, возможно будет удобнее использовать их в виде логических. Изменяем тип данных столбца `debt` на boolean И делаем проверку.

In [32]:
data['debt'] = data['debt'].astype('bool')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null bool
total_income        21524 non-null float64
purpose             21524 non-null object
years_employed      21524 non-null float64
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 2.6+ MB


**Вывод**

Изменён тип данных столбца `debt` на логический.

### Обработка дубликатов
<a id="duplicates"></a>

Ищем дублирующиеся строки методом duplicated():

In [33]:
print('duplicated =', data.duplicated().sum())

duplicated = 55


Удаляем дублирующиеся строки методом drop_duplicates(), одновременно обновляя индексы методом reset_index():

In [34]:
data = data.drop_duplicates().reset_index(drop=True)

Проверка, что дублирующиеся строки удалены:

In [35]:
print('duplicated =', data.duplicated().sum())

duplicated = 0


**Образование**

В колонке `education` записи сделаны в разном регистре, из-за этого можно было пропустить дубликаты. У этого столбца уже есть идентификаторы категорий, если они присвоены правильно, можно перенести значения в словарь и использовать только столбец с индикаторами. Проверим, что они приписаны без ошибок:

In [36]:
print(data.groupby('education_id')['education'].value_counts())
    

education_id  education          
0             высшее                  4710
              ВЫСШЕЕ                   273
              Высшее                   268
1             среднее                13704
              СРЕДНЕЕ                  772
              Среднее                  711
2             неоконченное высшее      667
              Неоконченное высшее       47
              НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
3             начальное                250
              НАЧАЛЬНОЕ                 17
              Начальное                 15
4             ученая степень             4
              УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
Name: education, dtype: int64


Идентификаторы приписаны правильно, а для каждой категории образования есть три варианта написания. 

Создаём новую таблицу без столбца `education` и словарь.

In [37]:
data_drop = data[['children', 'years_employed', 'dob_years', 'education_id', 'family_status', 
                  'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]

education_dict = data[['education', 'education_id']]


Удалим в словаре дублирующиеся строки и проверим, что всё правильно:

In [38]:
education_dict = education_dict.drop_duplicates('education_id').reset_index(drop=True)
print(education_dict)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       Ученая степень             4


Проверим, скрывались ли дубликаты в строках с неправильным регистром в записях об образовании:

In [39]:
print('duplicated =', data_drop.duplicated().sum())

duplicated = 17


Есть. Сразу проверим, не скрываются ли дубликаты в других столбцах. 

**Пол**

Мы уже удалили одну странную запись о гендере, так что здесь должно быть всё хорошо. Проверяем:

In [40]:
print(data_drop['gender'].value_counts())

F    14188
M     7281
Name: gender, dtype: int64


Так и есть.

**Семейный статус**

Проверка, есть ли дубли в столбце `family_status` (распечатываем список уникальных значений и проверяем, есть ли в них ошибки):

In [41]:
print(data_drop['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


Дублирующихся категорий нет, но категория 'Не женат / не замужем' записана с заглавной буквы. Заменяем значения при помощи функции str.lower():

In [42]:
data_drop['family_status'] = data_drop['family_status'].str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Pandas выдаёт сообщение о возможной ошибке. Сообщение часто бывает ложно-положительным, так что проверяем, сработал ли код в ячейке.  
Проверка, есть ли ошибки в `family_status_id`:

In [43]:
data_drop.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12344
1                 гражданский брак          4161
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 не женат / не замужем     2810
Name: family_status, dtype: int64

Оформляем словарь:


In [44]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
display(family_status_dict)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


С колонкой семейный статус всё в порядке. 

**Должность**

Проверяем колонку с должностями:

In [45]:
print(data_drop['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


С должностью всё в порядке. 

**Цель**

Целей кредита слишком много, чтобы проверить все, но, по крайней мере, можно проверить что нет проблем с регистром:

In [46]:
print(data_drop['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Ошибок, связанных с регистром, нет.

Удаляем дублирующиеся строки:

In [47]:
data_drop = data_drop.drop_duplicates().reset_index(drop=True)

Проверка:

In [48]:
print('duplicated =', data_drop.duplicated().sum())

duplicated = 0


**Вывод**

1. Удалены дублирующиеся строки.
2. Исправлены ошибки, связанные с регистром в столбце `family_status`. 
3. Для исправления ошибок, связанных с регистром в столбце `education` созданы две новые таблицы data_drop (со столбцом `education_id` и `education_dict`.

### Лемматизация
<a id="lemmas"></a>

Задача лемматизировать столбец `purpose`, чтобы выделить категории целей кредитов.

Загружаем библиотеку MyStem:

In [49]:
from pymystem3 import Mystem
m = Mystem() 

Пишем функцию для лемматизации строки. Функция должна принимать значение строки и возвращать список лемм.

In [50]:
def lemmas(str):
    # функция принимает как аргумент строку и возвращает список лемм
    lemmas = m.lemmatize(str)
    return lemmas

Проверка работы фунции:

In [51]:
lemmas('скоро весна, а на дворе столько снега')

['скоро',
 ' ',
 'весна',
 ', ',
 'а',
 ' ',
 'на',
 ' ',
 'двор',
 ' ',
 'столько',
 ' ',
 'снег',
 '\n']

Создаём в таблице data_drop столбец `lemmas`, в который записываем лемматизированные значения столбца `purpose`:

In [52]:
data_drop['lemmas'] = data_drop['purpose'].apply(lemmas)

Проверка:

In [53]:
display(data_drop.head())

,children,years_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,23.116912,42.0,0,женат / замужем,0,F,сотрудник,False,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,11.026860,36.0,1,женат / замужем,0,F,сотрудник,False,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,15.406637,33.0,1,женат / замужем,0,M,сотрудник,False,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,11.300677,32.0,1,женат / замужем,0,M,сотрудник,False,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,35.000000,53.0,1,гражданский брак,1,F,пенсионер,False,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

В таблице data_drop создан столбец `lemmas`, в который при помощи функции lemmas записаны лемматизированные цели.

### Категоризация данных
<a id="categories"></a>

В исследовании нужно ответить на вопросы, как связана вероятность не вернуть кредит вовремя с семейным положением, наличием детей, доходом и целями кредита. Для каждого столбца сформируем категории в соответствии с гипотезами.






**Цели кредита**
Гипотеза 1: Вовремя чаще возвращают небольшие и краткосрочные кредиты (например, на ремонт, на свадьбу или на автомобиль)

#### Категоризация целей
<a id="purpose_cat"></a>

Гипотеза 1: Вовремя чаще возвращают небольшие и краткосрочные кредиты (например, на ремонт, на свадьбу или на автомобиль).
Для проверки гипотезы нужно сформировать категории по целям.

Распечатаем уникальные значения столбца `purpose`, чтобы примерно оценить, какие категории было бы логично выделить:

In [54]:
print(data_drop['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Беглый просмотр позволяет выделить следующие цели: операции с недвижимостью, покупка автомобиля, получение образования, свадьба, ремонт. Создаём функцию, которая, принимая лемматизированные цели, распределит их по этим категориям по ключевым словам. То, что не будет содержать ключевые слова, функция определит как NA.

In [55]:
def categories(lemmas):
#     функция принимает на входе текстовую строку, 
#     проверяет, есть ли в ней какое-то из ключевых слов: 
#     [недвижимость, квартира, жилье, автомобиль, образование, свадьба, ремонт]
#     и возвращает категорию, к которой относится это ключевое слово.

    if 'образование' in lemmas:
        return 'образование'
    if 'ремонт'  in lemmas:
        return 'ремонт'
    if 'жилье' in lemmas or 'квартира' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'свадьба' in lemmas:
        return 'свадьба' 
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    else:
        return 'NA'
        

Проверка работы функции:

In [56]:
categories('хочу новая квартира')

'недвижимость'

In [57]:
categories('когда сделать ремонт')

'ремонт'

In [58]:
categories('свадьба')

'свадьба'

In [59]:
categories('абракадабра')

'NA'

Применяем функцию и создаём в таблице data_drop новый столбец `purpose_group` с категоризированными целями:

In [60]:
data_drop['purpose_group'] = data_drop['lemmas'].apply(categories)

Проверяем, что в категорию NA ничто не попало, а остальные цели распределились по категориям без ошибок:

In [61]:
display(data_drop.groupby('purpose_group')['purpose'].value_counts())

purpose_group  purpose                               
автомобиль     на покупку своего автомобиля              505
               автомобиль                                494
               сделка с подержанным автомобилем          486
               автомобили                                478
               на покупку подержанного автомобиля        478
               свой автомобиль                           478
               на покупку автомобиля                     471
               приобретение автомобиля                   461
               сделка с автомобилем                      455
недвижимость   операции с недвижимостью                  675
               покупка коммерческой недвижимости         661
               операции с жильем                         652
               покупка жилья для сдачи                   651
               операции с коммерческой недвижимостью     650
               жилье                                     646
               покупка жилья   

Удаляем вспомогательные столбцы `lemmas` и `purpose` и делаем проверку:

In [62]:
data_drop = data_drop.drop(['lemmas', 'purpose'], axis=1)
data_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 11 columns):
children            21452 non-null int64
years_employed      21452 non-null float64
dob_years           21452 non-null float64
education_id        21452 non-null int64
family_status       21452 non-null object
family_status_id    21452 non-null int64
gender              21452 non-null object
income_type         21452 non-null object
debt                21452 non-null bool
total_income        21452 non-null float64
purpose_group       21452 non-null object
dtypes: bool(1), float64(3), int64(3), object(4)
memory usage: 1.7+ MB


**Вывод**

Выделены 5 категорий целей кредита: недвижимость, автомобиль, образование, свадьба и ремонт. Для выделения категорий использована функция categories.

#### Категоризация доходов заёмщиков
<a id="income_cat"></a>

Гипотеза 1: Чем выше доход заёмщика, тем больше вероятность, что он вернёт доход вовремя.

Гипотеза 2: Вовремя чаще возвращают кредиты заёмщики со средним доходом.

Для анализа будет удобно разделить доходы клиентов на несколько групп, например, по квартилям.

Разобьём доходы заёмщиков на квартили. Находим первый квартиль, медиану и третий квартиль:

In [63]:
q1 = data_drop.quantile(q=0.25, axis=0, numeric_only=True, interpolation='linear')['total_income']
q2 = data_drop.quantile(q=0.5, axis=0, numeric_only=True, interpolation='linear')['total_income']
q3 = data_drop.quantile(q=0.75, axis=0, numeric_only=True, interpolation='linear')['total_income']


print('Q1 = ', q1)
print('Q2 = ', q2)
print('Q3 = ', q3)

Q1 =  107609.00883135476
Q2 =  144127.06075176556
Q3 =  196818.8008230424


Для разбивки числовых параметров на группы напишем функцию section. Она будет принимать на входе число и возвращать промежуток, в который это число попадает.

In [64]:
def section(count):
#     функция сравнивает входящее числовое значение с квартилями q1, q2 b q3 и возвращает промежуток 1, 2, 3 или 4
    if count <= q1:
        return 1
    if count <= q2:
        return 2
    if count <= q3:
        return 3
    else:
        return 4

Проверка работы функции:

In [65]:
print('888 - ', section(888))
print('120777 - ', section(120777))
print('177034 - ', section(177034))
print('2300000 - ', section(2300000))

888 -  1
120777 -  2
177034 -  3
2300000 -  4


Создадим в таблице ещё один столбец, в котором заёмщики будут отнесены к одной из групп при помощи функции section:

In [66]:
data_drop['income_id'] = data_drop['total_income'].apply(section)

**Вывод**

Заёмщики разделены на 4 равные группы по уровню дохода. Для разделения использована функция section.

#### Категоризация по наличию детей
<a id="children_cat"></a>

**Ещё немного предобработки**



В данных о детях были явные ошибки: '-1' и '20' детей. Скорее всего, '-1' возникло вместо '1', а '20' вместо '2'. Заменим сомнительные значения на более вероятные:

In [67]:
data_drop.loc[data_drop['children'] == -1, 'children'] = 1

data_drop.loc[data_drop['children'] == 20, 'children'] = 2

Проверка:

In [68]:
data_drop['children'].value_counts()

0    14089
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Категоризация**

*Гипотеза 1:* Вовремя чаще возвращают кредиты заёмщики, у которых нет детей совсем, так как они не обременены лишними расходами.

Для проверки этой гипотезы удобно будет разбить заёмщиков на две эти группы и создать со значениями "есть дети" / "нет детей"

*Гипотеза 2:* Чем больше у заёмщика детей, тем больше вероятность, что он не вернёт кредит вовремя.

*Гипотеза 3:* Наличие детей по разному влияет на вероятность вернуть кредит вовремя у заёмщиков в браке и не в в браке.

Для проверки гипотез 2 и 3 достаточно имеющихся данных.

Чтобы проанализировать зависимость от наличия детей, создадим столбец `has_children`, в котором будут логические значения.
Для этого напишем функцию iszero, которая будет численные значения переводить в текстовые. Если детей 0, функция будет возвращать строку "нет детей". Если детей больше нуля, возвращать строку "есть дети", если меньше нуля - NA.

In [69]:
def iszero(count):
    if count == 0:
        return 'нет детей'
    if count > 0:
        return 'есть дети'
    else:
        return 'NA'

Проверка работы функции:

In [70]:
print('135 =', iszero(135))
print('0 =', iszero(0))
print('-0 =', iszero(-0))
print('-18 =', iszero(-18))

135 = есть дети
0 = нет детей
-0 = нет детей
-18 = NA


Создаём столбец `has_children`:

In [71]:
data_drop['has_children'] = data_drop['children'].apply(iszero)

**Вывод**

Создали столбец `has_children`, в котором количество детей переведено в текстовое значение "есть дети" / "нет детей". Для создания столбца создали и использовали функцию iszero.

#### Категоризация по семейному статусу 
<a id="family_cat"></a>

*Гипотеза 1:* Вовремя чаще возвращают кредиты заёмщики, у которых есть семья (партнёр), так как им есть у кого попросить помощи, чесли что-то пойдёт не по плану.
Для проверки этой гипотезы создадим колонку `has_partner`, которая будет содержать текстовые значения "есть партнер" / "нет партнера"
    
*Гипотеза 2:* Вовремя чаще возвращают кредиты заёмщики в браке.
    
*Гипотеза 3:* Семейный статус по разному влияет на вероятность вернуть кредит вовремя у мужчин и у женщин.
    
Для проверки 2 и 3 гипотез все данные в удобном виде есть.

Пишем функцию has_partner которая принимает значения столбца `family_status_id` и возвращает строку "есть партнёр" (статусы 'женат / замужем' и 'гражданский брак') или "нет партнёра" (статусы 'вдовец / вдова', 'в разводе', 'не женат / не замужем').

In [72]:
def has_partner(count):
#     принимает численные значения столбца `family_status_id`,
# возвращает "есть партнёр" для значений 0 и 1 (статусы 'женат / замужем' и 'гражданский брак')
# и "нет партнёра" для значений 2, 3 и 4 (статусы 'вдовец / вдова', 'в разводе', 'не женат / не замужем'). 
# Если на входе оказываются значения, отличающиеся от ожидаемых, возвращает NA.
    if count == 0 or count == 1:
        return 'есть партнёр'
    if count == 2 or count == 3 or count == 4:
        return 'нет партнёра'
    else:
        return 'NA'

Проверка работы функции:

In [73]:
print('0 =', has_partner(0))
print('1 =', has_partner(1))
print('2 =', has_partner(2))
print('3 =', has_partner(3))
print('5 =', has_partner(5))
print('4 =', has_partner(4))

0 = есть партнёр
1 = есть партнёр
2 = нет партнёра
3 = нет партнёра
5 = NA
4 = нет партнёра


Применяем функцию has_partner для создания столбца `has_partner`:

In [74]:
data_drop['has_partner'] = data_drop['family_status_id'].apply(has_partner)
display(data_drop.head())

,children,years_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_group,income_id,has_children,has_partner
0,1,23.116912,42.0,0,женат / замужем,0,F,сотрудник,False,253875.639453,недвижимость,4,есть дети,есть партнёр
1,1,11.026860,36.0,1,женат / замужем,0,F,сотрудник,False,112080.014102,автомобиль,2,есть дети,есть партнёр
2,0,15.406637,33.0,1,женат / замужем,0,M,сотрудник,False,145885.952297,недвижимость,3,нет детей,есть партнёр
3,3,11.300677,32.0,1,женат / замужем,0,M,сотрудник,False,267628.550329,образование,4,есть дети,есть партнёр
4,0,35.000000,53.0,1,гражданский брак,1,F,пенсионер,False,158616.077870,свадьба,3,нет детей,есть партнёр


**ВЫвод**

Создан столбец `has_partner` с текстовыми значениями "есть партнер" / "нет партнера". Для создания столбца использована функция has_partner.

### Подготовка финальной таблицы для анализа
<a id="final_data"></a>

Распечатываем названия всех столбцов таблицы data_drop:

In [75]:
data_drop.columns

Index(['children', 'years_employed', 'dob_years', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose_group', 'income_id', 'has_children',
       'has_partner'],
      dtype='object')

Из них нам нужны категоризированные данные, касающиеся наличия задолженности (`debt`), наличия детей (`has_children` и `children`), семейного статуса (`has_partner` и `family_status`), пола (`gender`), цели кредита (`purpose_id`) и ежемесячного дохода (`income_id`). Создаёт таблицу data_final с нужными столбцами:

In [76]:
data_final = data_drop[[
    'debt',
    'has_children',
    'children',
    'has_partner',
    'family_status',
    'gender',
    'purpose_group',
    'income_id'
]]

data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 8 columns):
debt             21452 non-null bool
has_children     21452 non-null object
children         21452 non-null int64
has_partner      21452 non-null object
family_status    21452 non-null object
gender           21452 non-null object
purpose_group    21452 non-null object
income_id        21452 non-null int64
dtypes: bool(1), int64(2), object(5)
memory usage: 1.2+ MB


Распечатываем первые пять строк таблицы, чтобы проверить, что данные находятся в удобном для анализа виде.

In [77]:
display(data_final.head())

,debt,has_children,children,has_partner,family_status,gender,purpose_group,income_id
0,False,есть дети,1,есть партнёр,женат / замужем,F,недвижимость,4
1,False,есть дети,1,есть партнёр,женат / замужем,F,автомобиль,2
2,False,нет детей,0,есть партнёр,женат / замужем,M,недвижимость,3
3,False,есть дети,3,есть партнёр,женат / замужем,M,образование,4
4,False,нет детей,0,есть партнёр,гражданский брак,F,свадьба,3


**Вывод**

Таблица data_final готова и содержит в себе все необходимые сведения.

## Анализ данных
<a id="analise"></a>

### Зависимость между наличием детей и возвратом кредита в срок
<a id="children_factor"></a>

***Гипотеза 1:*** Вовремя чаще возвращают кредиты заёмщики, у которых есть семья (партнёр), так как им есть у кого попросить помощи, чесли что-то пойдёт не по плану.

Создаём сводную таблицу, в которой по строкам будет наличие детей, а по столбцам - частота возвращения и не возвращения кредита в срок.

In [78]:
data_pivot_has_children = data_final.pivot_table(
    index=['has_children'], 
    columns='debt', 
    values='family_status', 
    aggfunc='count')

display(data_pivot_has_children)

debt,False,True
has_children,,
есть дети,6685,678
нет детей,13026,1063


По абсолютным данным не очень удобно судить о соотношении надёжных и ненадёжных заёмщиков. Добавим к получившейся таблице столбцы с процентным соотношением надёжных и ненадёжных для каждой категории заёмщиков, отсортируем таблицу по убыванию надёжности и выведем столбцы `надёжные` и `ненадёжные` в процентах. Создадим для этого функцию to_percent.

In [79]:
def to_percent(data):
    data['надёжные'] = data[0] / (data[1] + data[0])
    data['ненадёжные'] = data[1] / (data[1] + data[0])
    data = data.sort_values(by='ненадёжные')
    data_style = data.style.format({'надёжные': "{:.1%}", 'ненадёжные': "{:.1%}"})   
    return data_style

display(to_percent(data_pivot_has_children))

debt,False,True,надёжные,ненадёжные
has_children,,,,
нет детей,13026,1063,92.5%,7.5%
есть дети,6685,678,90.8%,9.2%


Заёмщики, у которых нет детей, возвращают кредиты вовремя немного чаще, чем заёмщики с детьми. 

***Вывод по гипотезе 1:*** Заёмщики, у которых нет детей, действительно возвращают кредиты вовремя немного чаще, чем заёмщики с детьми.

***Гипотеза 2:*** Чем больше у заёмщика детей, тем больше вероятность, что он не вернёт кредит вовремя.

Создаём сводную таблицу для столбца `children` и применяем к ней функцию to_percent:

In [80]:
data_pivot_children = data_final.pivot_table(
    index=['children'], 
    columns='debt', 
    values='family_status', 
    aggfunc='count')

display(to_percent(data_pivot_children))

debt,False,True,надёжные,ненадёжные
children,,,,
0,13026,1063,92.5%,7.5%
3,303,27,91.8%,8.2%
1,4410,445,90.8%,9.2%
2,1926,202,90.5%,9.5%
4,37,4,90.2%,9.8%
5,9,nan,nan%,nan%


Наименее надёжными заёмщиками выглядят те, которых четверо детей, хотя для того, чтобы сделать достоверные выводы, их группа слишком маленькая. Более уверенно можно сделать вывод о том, что различия между группой заёмщиков, которых нет детей, и группой с детьми больше, чем между заёмщиками с разным количеством детей. Например, в группе с тремя детьми процент возвращённых вовремя кредитов оказался выше, чем в группе с двумя или одним ребёнком.

***Вывод по гипотезе 2:*** Количество детей, скорее всего, не существенно влияет на вероятность возвращения кредита в срок.

***Гипотеза 3:*** Наличие детей по разному влияет на вероятность вернуть кредит вовремя у заёмщиков в браке и не в в браке.

Создаём сводную таблицу, где по строкам будут сгруппированы значения столбцов `has_children` и `family_status`, а по столбцам - столбца `debt`. Применяем у таблице функцию to_percent.

In [81]:
data_pivot_children_married = data_final.pivot_table(
    index=['has_children', 'family_status'], 
    columns='debt', 
    values='gender', 
    aggfunc='count')

display(to_percent(data_pivot_children_married))

Действительно, наименее надёжными заёмщиками оказались те, у кого есть дети, но не оформлены отношения (гражданский брак или не женат / не замужем). Что логично.  
А наиболее надёжными заёмщиками - вдовы / вдовцы без детей, заёмщики в браке без детей и заёмщики в разводе (вне зависимости от наличия детей).

***Вывод по гипотезе 3:*** Анализ факторов наличия детей и семейного статуса вместе даёт интересный результат. Вероятность не вернуть кредит вовремя в группе заёмщиков, у которых есть дети, но не оформлены отношения (гражданский брак или не женат / не замужем) почти в два раза выше, чем в группах вдов / вдовцов без детей, заёмщиков в браке без детей и заёмщиков в разводе.

**Вывод о влиянии количества детей на надёжность заёмщика**

Наличие детей больше влияет на вероятность вернуть кредит вовремя, чем количество детей. Для разного количества детей различия не очень большие. Важным является сочетание факторов наличия детей и семейного статуса: наименее надёжны заёмщики, не состоящие в браке или состоящие в гражданском браке, но имеющие при этом детей. И наиболее надёжны заёмщики в группах вдов / вдовцов без детей, заёмщиков в браке без детей и заёмщиков в разводе (независимо от наличия детей).

### Зависимость между семейным положением и возвратом кредита в срок
<a id="family_factor"></a>

***Гипотеза 1:*** Вовремя чаще возвращают кредиты заёмщики, у которых есть семья (партнёр), так как им есть у кого попросить помощи, чесли что-то пойдёт не по плану.


Создаём свобную таблицу, в которой по строкам будут значения столбца `has_partner`, а по столбцам - столбца `debt`.
Применяем к ней функцию to_person и выводим на экран:

In [82]:
data_pivot_has_partner = data_final.pivot_table(
    index=['has_partner'], 
    columns='debt', 
    values='children', 
    aggfunc='count')

display(to_percent(data_pivot_has_partner))

debt,False,True,надёжные,ненадёжные
has_partner,,,,
есть партнёр,15169,1319,92.0%,8.0%
нет партнёра,4542,422,91.5%,8.5%


Разница между заёмщиками, у которых есть партнёр, и одинокими - всего 0,5%.

***Вывод по гипотезе 1:*** Гипотеза не подтвердилась, разница между заёмщиками, у которых есть партнёр, и одинокими совсем небольшая.

***Гипотеза 2:*** Вовремя чаще возвращают кредиты заёмщики в браке.

Создаём свобную таблицу для семейного статуса, применяем к ней функцию to_person и выводим на экран:

In [83]:
data_pivot_family = data_final.pivot_table(
    index=['family_status'], 
    columns='debt', 
    values='children', 
    aggfunc='count')

display(to_percent(data_pivot_family))

debt,False,True,надёжные,ненадёжные
family_status,,,,
вдовец / вдова,896,63,93.4%,6.6%
в разводе,1110,85,92.9%,7.1%
женат / замужем,11408,931,92.5%,7.5%
гражданский брак,3761,388,90.6%,9.4%
не женат / не замужем,2536,274,90.2%,9.8%


Самые надёжные заёмщики - вдовы и вдовцы, следом за ними идут те, что в разводе и женаты/замужем. А самые ненадёжные - неженатые/незамужние. Интересно, а среди незамужних есть разница между мужчинами и женщинами?

***Вывод по гипотезе 2:*** Различия по семейному статусу довольно существенные. Самые ненадёжные заёмщики - неженатые / незамужние и в гражданском браке. Самые надёжные - вдовы / вдовцы и заёмщики в разводе.

***Гипотеза 3:*** Семейный статус по разному влияет на вероятность вернуть кредит вовремя у мужчин и у женщин.

Создаём свобную таблицу, в которой по строкам будут значения столбцов `gender` и `family_status`, а по столбцам - столбца `debt`. Применяем к ней функцию to_person и выводим на экран:

In [84]:
data_pivot_family_gender = data_final.pivot_table(
    index=['gender', 'family_status'], 
    columns='debt', 
    values='children', 
    aggfunc='count')


display(to_percent(data_pivot_family_gender))


Судя по всему, мужчины в принципе намного менее надёжные заёмщики, чем женщины, то есть надёжность заёмщиков гораздо больше зависит от пола, чем от семейного положения. 

Что касается семейного положения, то его влияние на вероятность вовремя вернуть кредит разное для мужчин и для женщин.

Самые надёжные заёмщики - вдовы (5.8% задолженностей), женщины в разводе (6.5%), замужние и незамужние ( по 6.8%).
Следом идут женщины в гражданском браке, они самые ненадёжные среди женщин-заёмщиков (8.2%).
Все категории мужчин - менее надёжные заёмщики.
Самые надёжные среди мужчин - женатые (8.8%) и в разводе (9.3%).
За ними идут мужчины в гражданском браке (11.9%), неженатые (14.4%), и самые ненадёжные - вдовцы (20.0%).

***Вывод по гипотезе 3:*** Анализ двух факторов (семейный статус и пол) вместе показал, что для мужчин и для женщин семейный статус по-разному влияет на вероятность вернуть кредит вовремя. При анализе семейного статуса без учёта пола различия сглаживаются и приводят к ложным выводам, как в случае со вдовами и вдовцами.

**Вывод**

Семейный статус оказывает большое влияние на вероятность возврата кредита заёмщиком вовремя, однако его нельзя рассматривать в отрыве от пола.   
Так, например, вдовы и вдовцы находятся на противоположных полюсах, вдовы - самые надёжные заёмщики, а вдовцы - самые ненадёжные. У вдовцов вероятность не вернуть кредит вовремя в 3,4 раза больше, чем у вдов.  
В целом женщины гораздо чаще мужчин возвращают кредиты вовремя. Наиболее надёжные заёмщики - женщины, которые не состоят в отношениях (вдовы, женщины в разводе, незамужние).  
С мужчинами картина обратная: наименее надёжные заёмщики - одинокие мужчины и те, у кого отношения не оформлены официально (в гражданском браке - 11.9%, неженатые - 14.4% и вдовцы - 20.0%).

### Уровень дохода
<a id="income_factor"></a>

***Гипотеза 1:*** Чем выше доход заёмщика, тем больше вероятность, что он вернёт доход вовремя.  
***Гипотеза 2:*** Вовремя чаще возвращают кредиты заёмщики со средним доходом.

У нас есть две конкурирующие друг с другом гипотезы. Их можно проверить одновременно.
Создаём сводную таблицу, в которой по строкам сгруппированы значения столбца `income_id`, а по столбцам - столбца `debt`. Применяем к сводной таблице функцию to_percent и выводим её на экран.

In [85]:
data_pivot_income = data_final.pivot_table(
    index=['income_id'], 
    columns='debt', 
    values='children', 
    aggfunc='count')

display(to_percent(data_pivot_income))

debt,False,True,надёжные,ненадёжные
income_id,,,,
4,4916,377,92.9%,7.1%
1,4936,427,92.0%,8.0%
2,4905,458,91.5%,8.5%
3,4954,479,91.2%,8.8%


Самыми надёжными заёмщиками оказались клиенты с самыми высокими доходами (7.1% задолженностей). Следующая по надёжности - категория заёмщиков с низкими дохадами. Как ни странно, категории со средними доходами в среднем менее надёжны. Однако различия менду категориями по доходам не велики.

**Вывод**

Различия между группами с разным доходом не очень большие. В группе заёмщиков со средними доходами немного больше вероятность не вернуть кредит вовремя, чем в группе с высокими и низкими доходами.

###  Зависимость между целью кредита  и возвратом кредита в срок
<a id="purpose_factor"></a>

***Гипотеза 1:*** Вовремя чаще возвращают небольшие и краткосрочные кредиты (например, на ремонт, на свадьбу или на автомобиль)

Создаём сводную таблицу, в которой по строкам сгруппированы значения столбца `purpose_group`, а по столбцам - столбца `debt`. Применяем к сводной таблице функцию to_percent и выводим её на экран.

In [86]:
data_pivot_purpose = data_final.pivot_table(
    index=['purpose_group'], 
    columns='debt', 
    values='children', 
    aggfunc='count')

display(to_percent(data_pivot_purpose))

debt,False,True,надёжные,ненадёжные
purpose_group,,,,
ремонт,572,35,94.2%,5.8%
недвижимость,9456,747,92.7%,7.3%
свадьба,2137,186,92.0%,8.0%
образование,3643,370,90.8%,9.2%
автомобиль,3903,403,90.6%,9.4%


Гипотеза о том, что краткосрочные и небольшие кредиты возвращат лучше, не подтвердилась. Больше всего вовремя вовзращенных кредитов, действительно, в категории "ремонт", а меньше всего - в категории "автомобиль". Среди оставшихся категорий лучше возвращают кредиты на недвижимость и хуже возвращают кредиты на образование.

**Вывод**

Цель кредита довольно существенно влияет на вероятность, что он будет возвращён вовремя. Вероятность, что кредит будет возвращён вовремя больше, если его берут на ремонт или на недвижимость, и меньше, если его берут на образование или автомобиль.

## Вывод
<a id="Conclusion"></a>

**Краткий обзор проведённой работы**  

Данные подготовлены к работе. Удалены дублирующиеся данные, заполнены пропуски, исправлены ошибки.  
Выделены категории клиентов для анализа:  
   - для фактора "дети": наличие детей и количество детей  
   - для фактора "семейный статус": наличие партнёра и семейный статус  
   - для фактора "доход": 4 равные группы по уровню дохода  
   - для фактора "цели": проведена лемматизация и выделены группы клиентов по целям  
   
Построены сводные таблицы и описана вероятность возвращения кредита вовремя в исследуемых категориях клиентов.   

********

**Выводы**   

1. Наличие детей больше влияет на вероятность вернуть кредит вовремя, чем количество детей.  
   Для разного количества детей различия не очень большие.  
   Важным является сочетание факторов наличия детей и семейного статуса: наименее надёжны заёмщики, не состоящие в браке или состоящие в гражданском браке, но имеющие при этом детей. И наиболее надёжны заёмщики в группах вдов / вдовцов без детей, заёмщиков в браке без детей и заёмщиков в разводе (независимо от наличия детей).  
  
  
2. Семейное положение имеет большое влияние на вероятность возврата кредита заёмщиком вовремя, однако его нельзя рассматривать в отрыве от пола.   
   В целом женщины гораздо чаще мужчин возвращают кредиты вовремя. Наиболее надёжные заёмщики - женщины, которые не состоят в отношениях (вдовы, женщины в разводе, незамужние).  
   С мужчинами картина обратная: наименее надёжные заёмщики - одинокие мужчины и те, у кого отношения не оформлены официально (в гражданском браке, неженатые и вдовцы). На эти категории заёмщиков стоит обратить более пристальное внимание при принятии решения.  
  
  
3. Различия между группами с разным доходом не очень большие. В группе заёмщиков со средними доходами вероятность не вернуть кредит вовремя немного больше, чем в группе с высокими и низкими доходами.  
  
  
4. Цель кредита довольно существенно влияет на вероятность, что он будет возвращён вовремя. Вероятность, что кредит будет возвращён вовремя больше, если его берут на ремонт или на недвижимость, и меньше, если его берут на образование или автомобиль.

[В начало](#head)